In [756]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from textblob import TextBlob
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,LSTM
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints
import six
from tensorflow.keras.utils import deserialize_keras_object
from joblib import dump, load

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf
 
from tensorflow.keras.layers import concatenate, Lambda


In [757]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
''' The data from ASL model is added to other similar data source to increasee the data quantity'''

In [758]:
# data.head()

In [860]:
data = pd.read_csv('output.csv')


In [861]:
print("Data Shape",data.shape)      

Data Shape (2512, 2)


In [862]:
data.head()

,prediction,target_text
0,1795 westh st rerorore,2796 west golden willow drive
1,+39-313-5199,9734719887
2,998-121-1662,4977236992
3,elle cororerin,reallyloud.co.uksimaii
4,arilalallanges,kkaicd1.pixnet.net


In [863]:
# Define a dictionary called column_mapping that maps old column names to new column names.
column_mapping = {'prediction': 'SMS_TEXT', 'target_text': 'ENGLISH_TEXT'}

# Use the rename() function on the 'data' DataFrame to rename columns as specified in the column_mapping dictionary.
# The inplace=True parameter means that the changes are applied directly to the DataFrame 'data' without creating a new DataFrame.
data.rename(columns=column_mapping, inplace=True)


In [864]:
def preprocessing_steps(data):
    # Applying the length on both sms_text and english_text and filtering the sentences based on length 
    # adding start token and end token for inputs and output dataframe
    # '\t'-> start token which represents the start of the sentence
    # '\n'-> end token which represents the end of the sentence.
    # Removing the sms_length, english_length, and ENGLISH_TEXT and appending ENGLISH_INPUT, ENGLISH_OUTPUT for the decoder.
    
    data['sms_length'] = data['SMS_TEXT'].str.split().apply(len)
    data['eng_length'] = data['ENGLISH_TEXT'].str.split().apply(len)
    
    # Filtering based on sentence length
    data = data[data['sms_length'] <= 39]
    data = data[data['eng_length'] <= 40]
    
    # Adding start and end tokens
    data['ENGLISH_INPUT'] = '<start> ' + data['ENGLISH_TEXT'].astype(str)
    data['ENGLISH_OUTPUT'] = data['ENGLISH_TEXT'].astype(str) + ' <end>'
    
    # Dropping unnecessary columns
    data = data.drop(['sms_length', 'eng_length', 'ENGLISH_TEXT'], axis=1)
    
    return data


In [865]:
# Preprocess the data using preprocessing_steps function
preprocessed_data = preprocessing_steps(data)

# Print the shape of the preprocessed data
print(preprocessed_data.shape)

# Add '<end>' tag to the first English input
preprocessed_data.iloc[0]['ENGLISH_INPUT'] = str(preprocessed_data.iloc[0]['ENGLISH_INPUT']) + ' <end>'

# Commenting out the line that modifies the English output
# preprocessed_data.iloc[0]['ENGLISH_OUTPUT'] = '\t ' + str(preprocessed_data.iloc[0]['ENGLISH_OUTPUT']) + ' \n'


(2503, 3)


In [866]:
preprocessed_data.head()

,SMS_TEXT,ENGLISH_INPUT,ENGLISH_OUTPUT
0,1795 westh st rerorore,<start> 2796 west golden willow drive,2796 west golden willow drive <end>
1,+39-313-5199,<start> 9734719887,9734719887 <end>
2,998-121-1662,<start> 4977236992,4977236992 <end>
3,elle cororerin,<start> reallyloud.co.uksimaii,reallyloud.co.uksimaii <end>
4,arilalallanges,<start> kkaicd1.pixnet.net,kkaicd1.pixnet.net <end>


In [867]:
# Import the necessary module for splitting the data
from sklearn.model_selection import train_test_split

# Split the preprocessed_data into training and testing sets
train_data, test_data = train_test_split(preprocessed_data, test_size=0.01, random_state=42)

# Print the shape of the training and testing sets
print(train_data.shape)
print(test_data.shape)


(2477, 3)
(26, 3)


In [868]:
train_data

,SMS_TEXT,ENGLISH_INPUT,ENGLISH_OUTPUT
1136,Haven sleep yet? How have you been recently? H...,<start> Haven't slept yet? How have you been r...,Haven't slept yet? How have you been recently?...
2333,Hey u in e lt oredi?Im on my way..,"<start> Hey, are you in the LT already? I'm on...","Hey, are you in the LT already? I'm on my way...."
858,"No money to cut lor... Haha, no la, juz feel l...","<start> I have no money to cut. Haha, no, I ju...","I have no money to cut. Haha, no, I just feel ..."
929,Hi.. How was calculus? Feel that e lec is not ...,<start> Hi. How was calculus? Feel that the le...,Hi. How was calculus? Feel that the lecture is...
568,maintenance technician hw abt u?,<start> Maintenance technician. How about you?,Maintenance technician. How about you? <end>
...,...,...,...
1641,Anyone free today?Wanna go ecp?or expo?,<start> Anyone free today? Want to go to ecp? ...,Anyone free today? Want to go to ecp? or expo?...
1097,Do u have linear algebra past papers' sol??,<start> Do you have linear algebra past papers...,Do you have linear algebra past papers' soluti...
1132,Haha... Ü mean mich din join her meh?,<start> Haha. You mean Mich Din joins her?,Haha. You mean Mich Din joins her? <end>
1296,Hi its Kate how is your evening? I hope i can...,"<start> Hi, it's Kate, how is your evening? I ...","Hi, it's Kate, how is your evening? I hope I c..."


In [869]:
# Create a tokenizer for the 'SMS_TEXT' column
tokenizer = Tokenizer(filters=" ", char_level=False, lower=False)
print("SMS_TEXT")
# Fit the tokenizer on the 'SMS_TEXT' values in the training data
tokenizer.fit_on_texts(train_data['SMS_TEXT'].values)

# Create a tokenizer for the 'ENGLISH_INPUT' column
print("English text")
tokenizer_e = Tokenizer(filters=" ", char_level=False, lower=False)
# Fit the tokenizer on the 'ENGLISH_INPUT' values in the training data
tokenizer_e.fit_on_texts(train_data['ENGLISH_INPUT'].values)


SMS_TEXT
English text


In [870]:
# Fit the tokenizer on the English output data
tokenizer_e.fit_on_texts(train_data['ENGLISH_OUTPUT'].values)


In [871]:
# Calculate the vocabulary size for the encoder
encoder_vocab_size = len(tokenizer.word_index.keys())
print(encoder_vocab_size)

# Calculate the vocabulary size for the decoder
decoder_vocab_size = len(tokenizer_e.word_index.keys())
print(decoder_vocab_size)


7815
5464


In [872]:
tokenizer.word_index

{'u': 1,
 'to': 2,
 'i': 3,
 'the': 4,
 'I': 5,
 'a': 6,
 'at': 7,
 'me': 8,
 'go': 9,
 'my': 10,
 'ü': 11,
 'in': 12,
 'is': 13,
 'e': 14,
 'for': 15,
 'you': 16,
 'U': 17,
 'can': 18,
 'got': 19,
 'on': 20,
 'ur': 21,
 'of': 22,
 'not': 23,
 'are': 24,
 '2': 25,
 'n': 26,
 'so': 27,
 'Hey': 28,
 '4': 29,
 'So': 30,
 'or': 31,
 'will': 32,
 'we': 33,
 'time': 34,
 'be': 35,
 'have': 36,
 'it': 37,
 'wan': 38,
 'lor...': 39,
 "I'm": 40,
 'meet': 41,
 'r': 42,
 'and': 43,
 'do': 44,
 'no': 45,
 'if': 46,
 'all': 47,
 'dun': 48,
 'come': 49,
 'still': 50,
 'Ü': 51,
 'your': 52,
 'then': 53,
 'one': 54,
 'but': 55,
 'get': 56,
 'going': 57,
 'up': 58,
 'like': 59,
 'out': 60,
 'But': 61,
 'Hi': 62,
 'wat': 63,
 'b': 64,
 'how': 65,
 'ask': 66,
 'den': 67,
 'goin': 68,
 "i'm": 69,
 'home': 70,
 'this': 71,
 'with': 72,
 'when': 73,
 'call': 74,
 'w': 75,
 'think': 76,
 'Haha...': 77,
 'now': 78,
 'she': 79,
 'that': 80,
 'free': 81,
 'back': 82,
 'as': 83,
 'buy': 84,
 'la...': 85,
 'see':

In [873]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [874]:
# class Dataset:
#     def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
#         self.encoder_inps = data['italian'].values
#         self.decoder_inps = data['d_english_input'].values
#         self.decoder_outs = data['d_english_output'].values
#         self.tknizer_eng = tknizer_eng
#         self.tknizer_ita = tknizer_ita
#         self.max_len = max_len

#     def __getitem__(self, i):
#         self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
#         self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
#         self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])

#         self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
#         return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

#     def __len__(self): 
#         return len(self.encoder_inps)

    
# class Dataloder(tf.keras.utils.Sequence):    
#     def __init__(self, dataset, batch_size=1):
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.indexes = np.arange(len(self.dataset.encoder_inps))


#     def __getitem__(self, i):
#         start = i * self.batch_size
#         stop = (i + 1) * self.batch_size
#         data = []
#         for j in range(start, stop):
#             data.append(self.dataset[j])

#         batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
#         # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
#         return tuple([[batch[0],batch[1]],batch[2]])

#     def __len__(self):  # your model.fit_gen requires this function
#         return len(self.indexes) // self.batch_size

#     def on_epoch_end(self):
#         self.indexes = np.random.permutation(self.indexes)

# Attention

In [875]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self, inp_vocab_size, embedding_size, lstm_size, input_length):

        # Initialize Embedding layer
        # Initialize Encoder LSTM layer
        super().__init__()

        # Creating an Embedding layer for input vocabulary
        self.e_embed = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length)
        self.lstm_size = lstm_size
        
        # Creating an LSTM layer for the encoder
        self.e_lstm = LSTM(lstm_size, return_sequences=True, return_state=True)

    def call(self, input_sequence, states):
        '''
        This function takes a sequence input and the initial states of the encoder.
        Pass the input_sequence input to the Embedding layer, Pass the embedding layer output to encoder_lstm
        returns -- encoder_output, last time step's hidden and cell state
        '''
        # Passing the input sequence through the embedding layer
        embedding = self.e_embed(input_sequence)

        # Passing the embedded sequence through the LSTM layer
        output_state, enc_h, enc_c = self.e_lstm(embedding)

        return output_state, enc_h, enc_c

    def initialize_states(self, batch_size):
        '''
        Given a batch size it will return initial hidden state and initial cell state.
        If batch size is 32- Hidden state is zeros of size [32, lstm_units], cell state zeros is of size [32, lstm_units]
        '''
        # Initializing the initial hidden and cell states with zeros
        return [tf.zeros((batch_size, self.lstm_size)), tf.zeros((batch_size, self.lstm_size))]


In [876]:
class Attention(tf.keras.layers.Layer):
    '''
    Class that calculates scores based on the scoring_function using Bahdanau attention mechanism.
    '''
    def __init__(self, scoring_function, att_units):
        super(Attention, self).__init__()

        self.scoring_function = scoring_function

        if self.scoring_function == 'dot':
            # Initialize variables needed for Dot score function here
            self.dot = tf.keras.layers.Dot(axes=(1, 2))
            pass
        elif scoring_function == 'general':
            # Initialize variables needed for General score function here
            self.w = Dense(att_units)
            self.dot = tf.keras.layers.Dot(axes=(1, 2))
            pass
        elif scoring_function == 'concat':
            # Initialize variables needed for Concat score function here
            self.w1 = Dense(att_units)
            self.w2 = Dense(att_units)
            self.v = Dense(1)
            pass

    def call(self, decoder_hidden_state, encoder_output):
        '''
        Attention mechanism takes two inputs: current step -- decoder_hidden_state and all the encoder_outputs.
        * Based on the scoring function, we find the score or similarity between decoder_hidden_state and encoder_output.
        * Multiply the score function with your encoder_outputs to get the context vector.
        * Function returns context vector and attention weights (softmax - scores).
        '''
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state, 1)
        if self.scoring_function == 'dot':
            # Implement Dot score function here
            score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0, 2, 1)), encoder_output]), (0, 2, 1))
            pass
        elif self.scoring_function == 'general':
            # Implement General score function here
            mulpy = self.w(encoder_output)
            score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0, 2, 1)), mulpy]), (0, 2, 1),)
            pass
        elif self.scoring_function == 'concat':
            inte = self.w1(decoder_hidden_state) + self.w2(encoder_output)
            tan = tf.nn.tanh(inte)
            score = self.v(tan)

        att_weights = tf.nn.softmax(score, axis=1)
        context_vector = att_weights * encoder_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, att_weights


In [877]:
class OneStepDecoder(tf.keras.Model):
    def __init__(self, tar_vocab_size, embedding_dim, input_length, dec_units, score_fun, att_units):
        # Initialize decoder embedding layer, LSTM, and any other objects needed
        super(OneStepDecoder, self).__init__()
        self.dec_embed = Embedding(input_dim=tar_vocab_size, output_dim=embedding_dim)
        self.lstm = LSTM(dec_units, return_sequences=True, return_state=True)
        self.attention = Attention(scoring_function=score_fun, att_units=att_units)
        self.den = Dense(tar_vocab_size)

    def call(self, input_to_decoder, encoder_output, state_h, state_c):
        '''
        One step decoder mechanism step by step:
        A. Pass the input_to_decoder to the embedding layer and then get the output (batch_size, 1, embedding_dim)
        B. Using the encoder_output and decoder hidden state, compute the context vector.
        C. Concatenate the context vector with the output from Step A
        D. Pass the output from Step C to LSTM/GRU and get the decoder output and states (hidden and cell state)
        E. Pass the decoder output to the dense layer (vocab size) and store the result into output.
        F. Return the states from Step D, output from Step E, attention weights from Step B, context vector.
        '''
        embd = self.dec_embed(input_to_decoder)
        context_vec, attention_weights = self.attention(state_h, encoder_output)

        f_inp = tf.concat([tf.expand_dims(context_vec, 1), embd], axis=-1)
        # print(f_inp.shape)
        out, dec_h, dec_c = self.lstm(f_inp, [state_h, state_c])
        out = tf.reshape(out, (-1, out.shape[2]))
        output = self.den(out)

        return output, dec_h, dec_c, attention_weights, context_vec


In [878]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder, self).__init__()
      self.input_length = input_length
      self.out_vocab_size = out_vocab_size
      self.oneStepDecoder = OneStepDecoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
      self.out_vocab_size = out_vocab_size
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        # outputs = tf.TensorArray(dtype =  tf.float32, size= input_to_decoder.shape[1])
        outputs = tf.TensorArray(dtype =  tf.float32, size= tf.shape(input_to_decoder)[1])

        
        for timestep in range(tf.shape(input_to_decoder)[1]):

            output, decoder_hidden_state, decoder_cell_state, _, _ = self.oneStepDecoder(input_to_decoder[:, timestep:timestep+1],
                                                                                          encoder_output,decoder_hidden_state,decoder_cell_state)                                                                                            
                                                                                             
                                                                                             
            # Store the output in tensorarray
            outputs = outputs.write(timestep, output)
        # Return the tensor array
        outputs = tf.transpose(outputs.stack(), (1, 0, 2))
        return outputs
        
        
    

In [879]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,inp_vocab_size, out_vocab_size,input_length, enc_units,embedding_dim, dec_units, max_len, score_fun, att_units, batch_size):
    #Intialize objects from encoder decoder
    super(encoder_decoder, self).__init__()
    self.encoder = Encoder(inp_vocab_size= inp_vocab_size +1, embedding_size=embedding_dim, lstm_size= att_units,input_length= max_len)
    self.decoder = Decoder(out_vocab_size +1, embedding_dim, input_length, dec_units ,score_fun ,att_units)
    self.batch_size = batch_size

  
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    e_inp, d_inp = data[0], data[1]
    # print(data[0].shape)
    # print(data[1].shape)


    initial_state = self.encoder.initialize_states(self.batch_size)

    e_output, enc_h, enc_c = self.encoder(e_inp,initial_state)
    outputs = tf.TensorArray(dtype = tf.float32, size= 20)
        
    dec_h = enc_h
    dec_c = enc_c
    # print(dec_h.shape)
    output=self.decoder(d_inp,e_output, dec_h, dec_c)
  
    return output




<font color='blue'>**Custom loss function**</font>

In [880]:
# Define the Adam optimizer for training
optimizer = tf.keras.optimizers.Adam()

# Define the Sparse Categorical Crossentropy loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# Define a custom loss function that takes the targets and logits
def custom_lossfunction(targets, logits):
    # Create a mask to ignore loss for padded zeros
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    
    # Calculate the original loss
    loss_ = loss_object(targets, logits)
    
    # Apply the mask to the loss
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    # Calculate the mean of the loss considering only non-padded values
    return tf.reduce_mean(loss_)


<font color='blue'>**Training**</font>

Implement dot function here.

In [881]:
# Creating training and testing datasets using the 'Dataset' class
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 39, 40)
test_dataset = Dataset(test_data, tokenizer, tokenizer_e, 39, 40)

# Creating dataloaders for training and testing using the 'Dataloder' class
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)


In [882]:
# model = encoder_decoder(len(encoder_vocabluary), len(decoder_vocabluary),input_length, lstm_size, embedding_dim, att_units, max_len, 'dot', att_units, batch_size)
# model.compile(optimizer = 'Adam', loss = custom_lossfunction)



In [883]:
# Import the backend module from the tensorflow.keras library
from tensorflow.keras import backend as K

# Define a custom accuracy function that takes two arguments: y_true (true labels) and y_pred (predicted labels)
def accuracy(y_true, y_pred):

    # Convert the predicted labels into the format of the float32 data type
    pred_value = K.cast(K.argmax(y_pred, axis=-1), dtype='float32')
    
    # Compare the true labels with the predicted labels and convert the results into the format of the float32 data type
    true_value = K.cast(K.equal(y_true, pred_value), dtype='float32')

    # Create a mask that identifies the positions where the true labels are greater than 0 (where data is present)
    mask = K.cast(K.greater(y_true, 0), dtype='float32')
    
    # Calculate the sum of correctly predicted values by multiplying the mask with the true_value
    n_correct = K.sum(mask * true_value)
    
    # Calculate the total number of valid data points by summing the mask values
    n_total = K.sum(mask)
    
    # Return the ratio of correctly predicted values to the total number of valid data points as the accuracy
    return n_correct / n_total


In [884]:
# Setting the size of the LSTM layer
lstm_size = 100

# Setting the dimension of the embedding layer
embedding_dim = 128

# Defining the number of attention units
att_units = 100

# Setting the number of units in the decoder layer
dec_units = 100

# Setting the batch size for training
batch_size = 64


In [885]:
# Clear any previous session data
tf.keras.backend.clear_session()

# Create the model using the defined architecture
model2 = encoder_decoder(encoder_vocab_size, decoder_vocab_size, 39, lstm_size, embedding_dim, att_units, 40, 'concat', dec_units, batch_size)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(0.01)

# Compile the model with custom loss function and accuracy metric
model2.compile(optimizer=optimizer, loss=custom_lossfunction, metrics=[accuracy])

# Set the number of training and validation steps
train_steps = train_data.shape[0] // 64
valid_steps = test_data.shape[0] // 20

# Train the model
model2.fit(train_dataloader, steps_per_epoch=train_steps, epochs=30, validation_data=test_dataloader, validation_steps=valid_steps)


Epoch 1/30
38/38 [==============================] - 10s 210ms/step - loss: 2.3642 - accuracy: 0.0779 - val_loss: 1.3186 - val_accuracy: 0.1287
Epoch 2/30
38/38 [==============================] - 8s 207ms/step - loss: 2.2296 - accuracy: 0.0838 - val_loss: 1.3147 - val_accuracy: 0.1345
Epoch 3/30
38/38 [==============================] - 8s 208ms/step - loss: 2.1762 - accuracy: 0.0875 - val_loss: 1.2450 - val_accuracy: 0.1404
Epoch 4/30
38/38 [==============================] - 8s 210ms/step - loss: 2.0214 - accuracy: 0.1050 - val_loss: 1.1654 - val_accuracy: 0.1520
Epoch 5/30
38/38 [==============================] - 8s 208ms/step - loss: 1.8663 - accuracy: 0.1455 - val_loss: 1.0967 - val_accuracy: 0.1930
Epoch 6/30
38/38 [==============================] - 8s 210ms/step - loss: 1.7307 - accuracy: 0.1732 - val_loss: 1.0652 - val_accuracy: 0.1988
Epoch 7/30
38/38 [==============================] - 8s 209ms/step - loss: 1.6284 - accuracy: 0.1909 - val_loss: 1.0526 - val_accuracy: 0.2047
Epoch

In [ ]:
# rlp = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', factor=0.1, patience=5, verbose=10,
#     mode='auto', min_delta=0.00001, cooldown=0, min_lr=0.0001,
# )

## <font color='blue'>**Inference**</font>

<font color='blue'>**Plot attention weights**</font>

In [794]:
from math import log
from numpy import array
from numpy import argmax

# Beam search decoder function
def beam_search_decoder(data, k):
    # Initialize sequences with an empty sequence and score 0.0
    sequences = [[list(), 0.0]]
    
    # Loop through each row of data
    for row in data:
        all_candidates = list()
        
        # Loop through each sequence in sequences
        for i in range(len(sequences)):
            seq, score = sequences[i]
            
            # Loop through each element in the row
            for j in range(len(row)):
                try:
                    # Create a new candidate sequence and update the score
                    candidate = [seq + [j], score - log(row[j])]
                    all_candidates.append(candidate)
                except ValueError as e:
                    # If there's an exception, set the candidate score to 0
                    candidate = [seq + [j], 0]
                    all_candidates.append(candidate)
        
        # Order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        
        # Select the top k best candidates
        sequences = ordered[:k]
    
    return sequences

# Commented version of the code that explains each step


In [886]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Function to evaluate a given sentence
def evaluate(sentence):
    max_length_targ = 40
    max_length_inp  = 39
    
    # Initialize an array to store attention weights
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    # Tokenize the sentence using the tokenizer
    inputs = tokenizer.texts_to_sequences([sentence])
    inputs =  tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length_inp, padding='post') 
    inputs = tf.convert_to_tensor(inputs)
    result = '' 

    # Initialize the initial state of the decoder
    initial_state = model2.layers[0].initialize_states(batch_size=1)
    
    # Pass the inputs through the encoder
    encoder_outputs, state_h, state_c = model2.layers[0](inputs, initial_state)   
    dec_input = tf.expand_dims([tokenizer_e.word_index['<start>']], 0)

    # Loop through the decoder steps
    for i in range(max_length_targ):
        # Get the output, new states, attention weights, and result using oneStepDecoder
        Output, state_h, state_c, att_weights, _ = model2.layers[1].oneStepDecoder(dec_input, encoder_outputs, state_h, state_c)
        
        # Perform Beam Search Decoder
        Result_beam_list = beam_search_decoder(Output, k=1)
        Result_beam = Result_beam_list[0][0]

        # Reshape attention weights and store them
        attention_weights = tf.reshape(att_weights, (-1, ))
        attention_plot[i] = attention_weights.numpy()

        # Get the predicted ID using argmax
        predicted_id = tf.argmax(Output[0]).numpy()
  
        # Update the result string
        result += tokenizer_e.index_word[Result_beam[0]] + ' '
        
        # Check if the end token is predicted
        if tokenizer_e.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # Feed the predicted ID back into the model for the next step
        dec_input = tf.expand_dims([predicted_id], 0)
    
    return result, sentence, attention_plot


In [796]:
# This line retrieves the index of the "<end>" token in the word index of the "tokenizer_e" tokenizer.
tokenizer_e.word_index["<end>"]


2

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot attention
def plot_attention(attention, sentence, predicted_sentence):
    # Create a figure
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    
    # Display attention matrix as a heatmap
    ax.matshow(attention, cmap='viridis')
    
    # Set x-axis and y-axis labels using the provided sentences
    fontdict = {'fontsize': 14}
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    
    # Adjust ticks for clarity
    ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(mticker.MultipleLocator(1))
    
    # Display the plot
    plt.show()


In [727]:
def translate(sentence):
    # result, sentence, attention_plot = evaluate(sentence)
    # print('Input: %s' % (sentence))
    # print('Predicted translation: {}'.format(result))
    # print("-"*50)
    # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    # plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    # return result

    # Uncomment the lines below for translation
    result, sentence, attention_plot = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    print("-"*50)
    # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    # plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return result


In [ ]:
test_data.head()

In [ ]:
from tensorflow.keras.models import save_model


In [ ]:
# Specify the directory path to save the model
save_directory = "/Users/sayedraheel/Desktop/NLP/Sentence_correction"

# Instantiate the encoder_decoder model
tf.keras.models.save_model(model2, save_directory)


In [ ]:
save_directory = "/Users/sayedraheel/Desktop/NLP/Sentence_correction"

# Save the model
#model2.save(save_directory)
model2.save

In [ ]:
import os

In [ ]:
# Get the current working directory
current_directory = os.getcwd()


In [ ]:
current_directory

In [ ]:
# Translating the SMS_TEXT from the test_data dataframe at index 153
result = translate(test_data["SMS_TEXT"][153])


In [ ]:
# Translating the text from test_data at index 488
result = translate(test_data["SMS_TEXT"][488])


In [ ]:
# Accessing the word index of the token "you." in the tokenizer
tokenizer.word_index["you."]


In [ ]:
# Access the SMS_TEXT column from the test_data DataFrame
# This column contains the text messages that we want to analyze
sms_text_column = test_data["SMS_TEXT"]


In [ ]:
import nltk.translate.bleu_score as bleu
from tqdm import tqdm

# Define a function for calculating BLEU score
def BleuScore(validation):
    # Extract input and true output sentences from validation data
    input = list(validation['SMS_TEXT'])
    Y_true = list(validation['ENGLISH_OUTPUT'])
    
    results = []  # Store predicted results
    bleuscores = []  # Store BLEU scores
    input_sent_list = []  # Store input sentences
    
    # Loop through each input sentence
    for i in tqdm(range(len(input))):
        try:
            # Call the 'evaluate' function to get the prediction and attention plot
            result, sentence, attention_plot = evaluate(input[i])
            
            results.append(result)  # Store predicted result
            input_sent_list.append(sentence)  # Store input sentence
            
            # Calculate BLEU score for the predicted result and store it
            bleuscores.append(bleu.sentence_bleu(Y_true[i], result))
        except KeyError as e:
            pass  # Skip if there's a KeyError (missing key)
    
    # Calculate average BLEU score
    average_score = sum(bleuscores) / len(bleuscores)
    
    return average_score, bleuscores, results, input_sent_list

# Call the function to calculate BLEU scores
AvearageScore, bleuscores, results, input_sent_list = BleuScore(test_data)

# Print the average BLEU score
print("Average Bleuscore:", AvearageScore)


<font color='blue'>**Predict the sentence translation**</font>

In [887]:
def predict(input_sentence):
      '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <\t> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted character <\n>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         And get the character using the tokenizer(character index) and then store it in a string.
  E. Return the predicted sentence

  '''


    # Convert the input sentence into integer sequences using the previously used tokenizer
    input_sequence = tokenizer.texts_to_sequences([input_sentence])

    # Pad the input sequence to match the desired input length
    inputs = pad_sequences(input_sequence, maxlen=39, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    units = 100

    # Initialize hidden state for the encoder
    hidden = [tf.zeros((1, units))]

    # Pass the inputs through the encoder and get encoder output, hidden state, and cell state
    encoder_output, hidden_state, cell_state = model2.layers[0](inputs, hidden)
    dec_hidden = hidden_state

    # Initialize the decoder input with the start token
    dec_input = tf.expand_dims([tokenizer_e.word_index['<start>']], 0)

    # Loop through each time step of the decoder
    for t in range(40):
        # Use the decoder to generate predictions and update decoder hidden state and cell state
        predictions, dec_hidden, cell_state, attention_weights, context_vector = model2.layers[1].oneStepDecoder(dec_input, encoder_output, dec_hidden, cell_state)

        # Get the index of the predicted character with the highest probability
        predicted_id = tf.argmax(predictions[0]).numpy()

        # Convert the predicted index to the corresponding word and add it to the result string
        result += tokenizer_e.index_word[predicted_id] + ' '

        # Check if the end token has been predicted, and if so, return the result
        if tokenizer_e.word_index['<end>'] == predicted_id:
            return result

        # Prepare the predicted character for the next time step
        dec_input = tf.expand_dims([predicted_id], 0)

    return result


In [888]:
# Loop through each index in the test_data
for index in range(len(test_data)):
    print('*'*200)  # Print a separator line
    print("SMS_TEXT")  # Print a label for the SMS_TEXT
    print(test_data['SMS_TEXT'].values[index])  # Print the value of SMS_TEXT at the current index
    print("Prediction")  # Print a label for the prediction
    print(predict(test_data['SMS_TEXT'].values[index]))  # Print the prediction for the SMS_TEXT at the current index
    print("Original Sentence")  # Print a label for the original sentence
    print(test_data['ENGLISH_OUTPUT'].iloc[index])  # Print the value of ENGLISH_OUTPUT at the current index


********************************************************************************************************************************************************************************************************
SMS_TEXT
i tink i get her flowers,hee... or smting... i wun stay long lar.
Prediction
Haha. Okay. Are you going to be going for the patio <end> 
Original Sentence
I think I get her flowers. Or something. I want to stay long. <end>
********************************************************************************************************************************************************************************************************
SMS_TEXT
Dinner changed to tm becos xy can't make it on wednesday.
Prediction
Hello Peach! My Og going. They should be going. They don't want to go out. <end> 
Original Sentence
Dinner have been changed to tomorrow because Xy can't make it on Wednesday. <end>
***********************************************************************************************************

In [ ]:
import nltk.translate.bleu_score as bleu

# Initialize a variable to keep track of the total BLEU score
sum = 0

# Iterate through each data point in the test_data
for index in range(len(test_data)):
    # Get the reference (ground truth) sentence from the test_data
    reference = test_data['ENGLISH_OUTPUT'].iloc[index]
    
    # Generate a translation using the predict function
    translation = predict(test_data['SMS_TEXT'].iloc[index])
    
    # Calculate the BLEU score for the generated translation compared to the reference
    sum += bleu.sentence_bleu(reference, translation)

# Calculate the average BLEU score by dividing the sum by the number of data points
average_bleu = sum / len(test_data)
print(average_bleu)


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6780770304835106
